<i> Note: This notebook is inspired by the Topic-Modeling-Latent-Dirichlet-Allocation series at: https://github.com/rhasanbd/Topic-Modeling-Latent-Dirichlet-Allocation </i>

## Latent Dirichlet Allocation - Implementation on Yelp dataset

In this notebook, we implement Latent Dirichlet Allocation(LDA) on the Yelp reviews data to carry out Topic Modelling. We use the Gensim topic modelling API https://radimrehurek.com/gensim/models/ldamodel.html. Scikit-Learn implementation is also available (we use Gensim since it provides more functionality and application like Topic Coherence Pipeline or Dynamic Topic Modeling.)

We build an **end-to-end Natural Language Processing (NLP) pipeline**, starting with raw data and running through preparing, modeling, visualization.
The steps that we will carry out involves the following:
1. Exploratory Data Analysis
2. Data Cleaning and Pre-processing
3. Topic modeling with LDA
4. Determine optimal number of Topics
5. Visualize topic model using pyLDAvis

### Yelp Review Dataset
The Yelp Review Dataset is a CSV file that contains a sub-sample of 10,000 reviews extracted from the Yelp dataset available at: https://www.yelp.com/dataset.

The review dataset contains the following fields:
- business_id : Unique identifier of business
- date : Data of review posted YYYY-MM-DD
- review_id : Unique identifier of review
- stars : Star rating (upto 4 stars)
- text : Review text
- user_id : Unique identifier of user who posted the review
- cool : Number of cool votes received
- useful : Number of useful votes received
- funny : Number of funny votes received

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

%pylab inline
import pandas as pd
import pickle as pk
from scipy import sparse as sp

import nltk
nltk.download('wordnet')

2020-03-14 21:24:31,028 : DEBUG : $HOME=C:\Users\rojin
2020-03-14 21:24:31,030 : DEBUG : CONFIGDIR=C:\Users\rojin\.matplotlib
2020-03-14 21:24:31,031 : DEBUG : matplotlib data path: c:\users\rojin\appdata\local\programs\python\python37\lib\site-packages\matplotlib\mpl-data
2020-03-14 21:24:31,034 : DEBUG : loaded rc file c:\users\rojin\appdata\local\programs\python\python37\lib\site-packages\matplotlib\mpl-data\matplotlibrc
2020-03-14 21:24:31,039 : DEBUG : matplotlib version 3.1.2
2020-03-14 21:24:31,039 : DEBUG : interactive is False
2020-03-14 21:24:31,040 : DEBUG : platform is win32
2020-03-14 21:24:31,040 : DEBUG : loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'encodings.cp1252', 'site', '

2020-03-14 21:24:31,085 : DEBUG : CACHEDIR=C:\Users\rojin\.matplotlib
2020-03-14 21:24:31,089 : DEBUG : Using fontManager instance from C:\Users\rojin\.matplotlib\fontlist-v310.json
2020-03-14 21:24:31,195 : DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-03-14 21:24:31,200 : DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-03-14 21:24:31,202 : DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rojin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Load & Explore the Data

In [2]:
df = pd.read_csv('Data/yelp_academic_dataset_review_10000.csv')

# df1 = pd.read_csv('/Users/hasan/datasets/NIPS2015_Papers.csv')

df.head()

,business_id,date,review_id,stars,text,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,1/26/2011,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,7/27/2011,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,6/14/2012,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,5/27/2010,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,1/5/2012,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


## Description of the Data

DataFrame’s info() method is useful to get a quick description of the data, in particular the total number of rows, and each attribute’s type and number of non-null values.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
business_id    10000 non-null object
date           10000 non-null object
review_id      10000 non-null object
stars          10000 non-null int64
text           10000 non-null object
user_id        10000 non-null object
cool           10000 non-null int64
useful         10000 non-null int64
funny          10000 non-null int64
dtypes: int64(4), object(5)
memory usage: 703.2+ KB


## Dimension the Data

Get the dimension (number of rows and columns) of the data using DataFrame's shape method.

In [4]:
print("Dimension of the data: ", df.shape)

no_of_rows = df.shape[0]
no_of_columns = df.shape[1]

print("No. of Rows: %d" % no_of_rows)
print("No. of Columns: %d" % no_of_columns)

Dimension of the data:  (10000, 9)
No. of Rows: 10000
No. of Columns: 9


## Convert the DataFrame Object into a 2D Array of Documents

We convert the documents from DataFrame object to an array of documents.

It's a 2D array in which each row reprents a document.

In [5]:
docs_array = array(df['text'])

print("Dimension of the documents array: ", docs_array.shape)

# Display the first document
#print(docs_array[0])

Dimension of the documents array:  (10000,)


## Pre-process the Data

Pre-processing of the text data is done using the following steps:

- Convert to lowercase 
- Tokenize (split the documents into tokens or words)
- Remove numbers, but not words that contain numbers
- Remove words that are less than or two characters
- Lemmatize the tokens/words


### Tokenization

We tokenize the text using a regular expression tokenizer from NLTK. We remove numeric tokens and tokens that are only a single character, as they don’t tend to be useful, and the dataset contains a lot of them.


The NLTK Regular-Expression Tokenizer class "RegexpTokenizer" splits a string into substrings using a regular expression. We use the regular expression "\w+" to matche token of words. 

See the following two links for a list of regular expressions and NLTK tokenize module.
https://github.com/tartley/python-regex-cheatsheet/blob/master/cheatsheet.rst
https://www.nltk.org/api/nltk.tokenize.html


## Function to Convert the 2D Document Array into a 2D Array of Tokenized Documents

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+') # Tokenize the words.
    
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 4] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

## Convert the 2D Document Array into a 1D Array of Tokenized Words

In [7]:
# Convert the 2D Document Array into a 1D Array of Tokenized Words
%time docs = docs_preprocessor(docs_array)

Wall time: 4 s


In [8]:
print("Length of the 2D Array of Tokenized Documents: ", len(docs))

#Display the first two document
print(docs[0:2])

Length of the 2D Array of Tokenized Documents:  10000
[['birthday', 'breakfast', 'excellent', 'weather', 'perfect', 'which', 'sitting', 'outside', 'overlooking', 'their', 'ground', 'absolute', 'pleasure', 'waitress', 'excellent', 'arrived', 'quickly', 'saturday', 'morning', 'looked', 'place', 'fill', 'pretty', 'quickly', 'earlier', 'better', 'yourself', 'favor', 'their', 'bloody', 'phenomenal', 'simply', 'pretty', 'ingredient', 'their', 'garden', 'blend', 'fresh', 'order', 'amazing', 'while', 'everything', 'look', 'excellent', 'white', 'truffle', 'scrambled', 'vegetable', 'skillet', 'tasty', 'delicious', 'piece', 'their', 'griddled', 'bread', 'amazing', 'absolutely', 'complete', 'toast', 'anyway'], ['people', 'review', 'about', 'place', 'please', 'everyone', 'probably', 'griping', 'about', 'something', 'their', 'fault', 'there', 'people', 'friend', 'arrived', 'about', 'sunday', 'pretty', 'crowded', 'thought', 'sunday', 'evening', 'thought', 'would', 'forever', 'seated', 'come', 'seatin

## Compute Bigrams/Trigrams:


When topics are very similar, we may **use phrases** rather than single/individual words to distinguis each topic. 

Thus, we compute both bigrams and trigrams. Depending on the dataset it may not be necessary to create trigrams.

Note that we only keep the **frequent** phrases (bigrams/trigrams).

#### Bigrams
Bigrams are sets of two adjacent words. Using bigrams we can get phrases like “machine_learning” in our output (spaces are replaced with underscores). Without bigrams we would only get “machine” and “learning”.

Note that in the code below, we find bigrams and then add them to the original data, because we would like to keep the words “machine” and “learning” as well as the bigram “machine_learning”.

In [9]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 1000 times or more).
bigram = Phrases(docs, min_count=70)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a trigram, add to document.
            docs[idx].append(token)

2020-03-14 21:24:37,017 : INFO : collecting all words and their counts
2020-03-14 21:24:37,018 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-03-14 21:24:37,776 : INFO : collected 328444 word types from a corpus of 461593 words (unigram + bigrams) and 10000 sentences
2020-03-14 21:24:37,776 : INFO : using 328444 counts as vocab in Phrases<0 vocab, min_count=70, threshold=10.0, max_vocab_size=40000000>
2020-03-14 21:24:37,777 : INFO : collecting all words and their counts
2020-03-14 21:24:37,778 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-03-14 21:24:40,335 : INFO : collected 329543 word types from a corpus of 459313 words (unigram + bigrams) and 10000 sentences
2020-03-14 21:24:40,336 : INFO : using 329543 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


## Remove Rare and Common Tokens/Words

We remove rare words and common words based on their document frequency. 

For example, we may remove words that appear in less than 10 documents or in more than 20% of the documents. 

In [10]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 100 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=70, no_above=0.60) #100,60

print('Number of unique words after removing rare and common words:', len(dictionary))

2020-03-14 21:24:43,839 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-03-14 21:24:44,683 : INFO : built Dictionary(23478 unique tokens: ['absolute', 'absolutely', 'amazing', 'anyway', 'arrived']...) from 10000 documents (total 487417 corpus positions)
2020-03-14 21:24:44,722 : INFO : discarding 22450 tokens: [('absolute', 55), ('blend', 44), ('bloody', 46), ('favor', 51), ('fill', 15), ('griddled', 3), ('overlooking', 11), ('phenomenal', 58), ('pleasure', 57), ('saturday_morning', 25)]...
2020-03-14 21:24:44,723 : INFO : keeping 1028 tokens which were in no less than 70 and no more than 6000 (=60.0%) documents
2020-03-14 21:24:44,730 : DEBUG : rebuilding dictionary, shrinking gaps
2020-03-14 21:24:44,732 : INFO : resulting dictionary: Dictionary(1028 unique tokens: ['absolutely', 'amazing', 'anyway', 'arrived', 'better']...)


Number of unique words in initital documents: 23478
Number of unique words after removing rare and common words: 1028


## Bag-of-Words Representation of Data


Finally, we transform the documents to a **vectorized form**. 

We simply compute the frequency of each word, including the bigrams.

In [11]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 1028
Number of documents: 10000


In [12]:
for i in range (0 , 200):
    print(dictionary[i])

absolutely
amazing
anyway
arrived
better
birthday
bread
breakfast
complete
delicious
earlier
everything
excellent
fresh
garden
ground
ingredient
look
looked
morning
order
outside
perfect
piece
place
pretty
quickly
saturday
simply
sitting
tasty
their
toast
vegetable
waitress
weather
which
while
white
yourself
about
awesome
baked
because
come
crowded
decided
drink
evening
everyone
forever
friend
great
issue
liked
people
personal
pizza
placed
pleasant
please
price
probably
review
reviewer
sauce
seated
seating
serious
server
shared
small
someone
something
sunday
there
these
thing
thought
waiter
wanted
would
candy
plate
selection
clean
convenient
desert
located
scottsdale
wonderful
always
customer
detail
general
important
manager
mistake
satisfied
speak
staff
state
surprised
thanks
totally
treat
after
almost
another
apparently
apple
beautiful
before
bring
butter
couple
crisp
definitely
dessert
enough
entree
explain
fairly
feeling
finished
forgot
glass
honey
impressed
including
inside
kitche

## Training the LDA Model

We use the gensim.models.LdaModel class for performing LDA.

We need to set the parameters of the LdaModel object carefully. The full list of the parameters are given:

https://radimrehurek.com/gensim/models/ldamodel.html


#### Below we discuss the setting of some of the key parameters.

- num_topics (int, optional) – The number of requested latent topics to be extracted from the training corpus.

 
LDA is an unsupervised technique, meaning that we don't know prior to running the model how many topics exits in our corpus. It depends on the data and the application. We may use the following two technique to determine the number of topics.


        Technique 1: Topic Coherence 
The main technique to determine the number of topics is **Topic coherence**:
http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf


        Technique 2: Visualizing Inter-Topic Distance 
Use the LDA visualization tool pyLDAvis to observe Intertopic Distance Map (discussed later). By varying the number of topics we could determine the optimal value from the visualization.

We **use both techniques** to determine the optimal number of topics.


- chunksize (int, optional) – Number of documents to be used in each training chunk.

It controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. 

We set chunksize = 2000, which is more than the amount of documents. Thus, it processes all the data in one go. 

Chunksize can however influence the quality of the model.


- passes (int, optional) – Number of passes through the corpus during training.

It controls how often we train the model on the entire corpus. Another word for passes might be “epochs”. 


- iterations (int, optional) – Maximum number of iterations through the corpus when inferring the topic distribution of a corpus.

It is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. 

        It is important to set the number of “passes” and “iterations” high enough.



#### How to Set "passes" and "iterations":

First, enable logging and set eval_every = 1 (however, it might slow down, so, we use None) in LdaModel. 

When training the model look for a line in the log that looks something like this:

        2020-02-25 19:07:04,716 : DEBUG : 49/403 documents converged within 400 iterations

If we set passes = 20, we will see this line 20 times. 

### Important: We need to make sure that by the final passes, most of the documents have converged. 

For example, if passes = 20 and iterations = 400, then, we should see something like following:


        2020-02-25 19:07:18,041 : INFO : PROGRESS: pass 19, at document #403/403
        2020-02-25 19:07:18,042 : DEBUG : performing inference on a chunk of 403 documents
        2020-02-25 19:07:18,627 : DEBUG : 402/403 documents converged within 400 iterations

Thus, want to choose both passes and iterations to be high enough for this to happen.


- eval_every (int, optional) – Log perplexity is estimated every that many updates. Setting this to 1 slows down training by ~2x.


- alpha ({numpy.ndarray, str}, optional): Can be set to an 1D array of length equal to the number of expected topics that expresses our a-priori belief for the each topics’ probability. 

Alternatively default prior selecting strategies can be employed by supplying a string:

        ’asymmetric’: Uses a fixed normalized asymmetric prior of 1.0 / topicno.

        ’auto’: Learns an asymmetric prior from the corpus (not available if distributed==True).
        
        
- eta ({float, np.array, str}, optional) – A-priori belief on word probability.

It can be:

        scalar for a symmetric prior over topic/word probability,

        vector of length num_words to denote an asymmetric user defined probability for each word,

        matrix of shape (num_topics, num_words) to assign a probability for each word-topic combination,

        the string ‘auto’ to learn the asymmetric prior from the data.


We set alpha = 'auto' and eta = 'auto'. Again this is somewhat technical, but essentially we are automatically learning two parameters in the model that we usually would have to specify explicitly.

In [13]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 13 #11

chunksize = 500 # Size of the doc looked at every pass
passes = 100 # Number of passes through documents
iterations = 1000 # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus.
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

2020-03-14 21:24:45,040 : INFO : using autotuned alpha, starting with [0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308]
2020-03-14 21:24:45,041 : INFO : using serial LDA version on this node
2020-03-14 21:24:45,044 : INFO : running online (multi-pass) LDA training, 13 topics, 100 passes over the supplied corpus of 10000 documents, updating model once every 500 documents, evaluating perplexity every 0 documents, iterating 1000x with a convergence threshold of 0.001000
2020-03-14 21:24:45,044 : INFO : PROGRESS: pass 0, at document #500/10000
2020-03-14 21:24:45,045 : DEBUG : performing inference on a chunk of 500 documents
2020-03-14 21:24:45,669 : DEBUG : 498/500 documents converged within 1000 iterations
2020-03-14 21:24:45,671 : INFO : optimized alpha [0.07100186, 0.058656663, 0.0588093, 0.0639131, 0.058116876, 0.06294325, 0.05986054, 0.05795241, 0.061356787, 0.057069287, 0.05896

2020-03-14 21:24:46,963 : INFO : merging changes from 500 documents into a model of 10000 documents
2020-03-14 21:24:46,965 : INFO : topic #9 (0.054): 0.033*"great" + 0.026*"service" + 0.025*"carne_asada" + 0.025*"place" + 0.016*"lunch" + 0.016*"during" + 0.016*"little" + 0.016*"think" + 0.015*"definitely" + 0.015*"group"
2020-03-14 21:24:46,966 : INFO : topic #7 (0.058): 0.030*"their" + 0.025*"salsa" + 0.025*"taco" + 0.020*"mexican" + 0.019*"place" + 0.018*"chip_salsa" + 0.018*"burrito" + 0.016*"cheese" + 0.016*"really" + 0.014*"chip"
2020-03-14 21:24:46,966 : INFO : topic #5 (0.073): 0.026*"customer_service" + 0.024*"place" + 0.022*"service" + 0.020*"their" + 0.017*"customer" + 0.015*"there" + 0.011*"business" + 0.010*"reasonably_priced" + 0.010*"local" + 0.010*"about"
2020-03-14 21:24:46,966 : INFO : topic #3 (0.083): 0.035*"pizza" + 0.031*"there" + 0.018*"great" + 0.017*"place" + 0.015*"always" + 0.014*"people" + 0.014*"their" + 0.012*"never" + 0.011*"would" + 0.009*"really"
2020-0

2020-03-14 21:24:48,129 : INFO : topic diff=0.119238, rho=0.333333
2020-03-14 21:24:48,131 : INFO : PROGRESS: pass 0, at document #5000/10000
2020-03-14 21:24:48,132 : DEBUG : performing inference on a chunk of 500 documents
2020-03-14 21:24:48,384 : DEBUG : 500/500 documents converged within 1000 iterations
2020-03-14 21:24:48,387 : INFO : optimized alpha [0.116575554, 0.06994644, 0.0713157, 0.097537845, 0.072367035, 0.08123131, 0.07059344, 0.05637258, 0.07880686, 0.053682353, 0.070336856, 0.069977164, 0.062393196]
2020-03-14 21:24:48,388 : DEBUG : updating topics
2020-03-14 21:24:48,389 : INFO : merging changes from 500 documents into a model of 10000 documents
2020-03-14 21:24:48,391 : INFO : topic #9 (0.054): 0.047*"carne_asada" + 0.044*"great" + 0.032*"service" + 0.032*"place" + 0.019*"definitely" + 0.019*"lunch" + 0.018*"little" + 0.017*"carne" + 0.016*"during" + 0.016*"asada"
2020-03-14 21:24:48,393 : INFO : topic #7 (0.056): 0.033*"their" + 0.031*"burrito" + 0.030*"mexican" + 0

2020-03-14 21:24:49,543 : INFO : topic #8 (0.088): 0.027*"table" + 0.023*"would" + 0.018*"minute" + 0.016*"there" + 0.014*"about" + 0.014*"after" + 0.013*"asked" + 0.013*"order" + 0.011*"which" + 0.010*"ordered"
2020-03-14 21:24:49,545 : INFO : topic #3 (0.108): 0.046*"pizza" + 0.028*"there" + 0.022*"place" + 0.018*"always" + 0.017*"their" + 0.015*"great" + 0.011*"people" + 0.010*"really" + 0.010*"time" + 0.010*"never"
2020-03-14 21:24:49,546 : INFO : topic #0 (0.137): 0.016*"place" + 0.013*"chicken" + 0.013*"which" + 0.012*"ordered" + 0.011*"salad" + 0.011*"sauce" + 0.010*"their" + 0.010*"really" + 0.010*"cheese" + 0.010*"there"
2020-03-14 21:24:49,547 : INFO : topic diff=0.103729, rho=0.267261
2020-03-14 21:24:49,549 : INFO : PROGRESS: pass 0, at document #7500/10000
2020-03-14 21:24:49,550 : DEBUG : performing inference on a chunk of 500 documents
2020-03-14 21:24:49,812 : DEBUG : 500/500 documents converged within 1000 iterations
2020-03-14 21:24:49,816 : INFO : optimized alpha [0.

2020-03-14 21:24:51,089 : DEBUG : updating topics
2020-03-14 21:24:51,091 : INFO : merging changes from 500 documents into a model of 10000 documents
2020-03-14 21:24:51,093 : INFO : topic #7 (0.056): 0.043*"taco" + 0.043*"mexican" + 0.042*"chip" + 0.038*"salsa" + 0.035*"chip_salsa" + 0.034*"burrito" + 0.030*"their" + 0.025*"bean" + 0.025*"place" + 0.019*"fresh"
2020-03-14 21:24:51,094 : INFO : topic #9 (0.056): 0.069*"great" + 0.045*"service" + 0.036*"place" + 0.028*"carne_asada" + 0.021*"during" + 0.021*"definitely" + 0.019*"atmosphere" + 0.017*"really" + 0.017*"happy" + 0.015*"lunch"
2020-03-14 21:24:51,095 : INFO : topic #8 (0.103): 0.024*"table" + 0.022*"would" + 0.020*"minute" + 0.018*"there" + 0.016*"after" + 0.015*"about" + 0.014*"order" + 0.014*"asked" + 0.012*"another" + 0.011*"before"
2020-03-14 21:24:51,096 : INFO : topic #3 (0.119): 0.044*"pizza" + 0.027*"there" + 0.024*"place" + 0.022*"always" + 0.016*"their" + 0.013*"great" + 0.011*"time" + 0.010*"people" + 0.009*"never"

...

2020-03-14 21:28:14,842 : DEBUG : updating topics
2020-03-14 21:28:14,843 : INFO : merging changes from 500 documents into a model of 10000 documents
2020-03-14 21:28:14,846 : INFO : topic #9 (0.114): 0.183*"carne_asada" + 0.106*"pleasant" + 0.076*"carne" + 0.066*"asada" + 0.060*"environment" + 0.059*"carrot" + 0.056*"platter" + 0.056*"cover" + 0.054*"satisfied" + 0.052*"relaxing"
2020-03-14 21:28:14,847 : INFO : topic #12 (0.144): 0.192*"staff_friendly" + 0.114*"hotel" + 0.110*"saturday_night" + 0.082*"downtown" + 0.069*"staff" + 0.054*"night" + 0.050*"starbucks" + 0.049*"stayed" + 0.048*"friendly" + 0.047*"room"
2020-03-14 21:28:14,848 : INFO : topic #0 (3.960): 0.028*"restaurant" + 0.027*"which" + 0.022*"chicken" + 0.022*"ordered" + 0.019*"salad" + 0.018*"cheese" + 0.018*"sauce" + 0.017*"delicious" + 0.017*"lunch" + 0.016*"fresh"
2020-03-14 21:28:14,849 : INFO : topic #4 (3.964): 0.106*"place" + 0.081*"there" + 0.049*"really" + 0.026*"thing" + 0.026*"pretty" + 0.025*"drink" + 0.025*

2020-03-14 21:28:15,250 : INFO : topic diff=0.007962, rho=0.091287
2020-03-14 21:28:15,254 : INFO : PROGRESS: pass 99, at document #5500/10000
2020-03-14 21:28:15,255 : DEBUG : performing inference on a chunk of 500 documents
2020-03-14 21:28:15,328 : DEBUG : 500/500 documents converged within 1000 iterations
2020-03-14 21:28:15,332 : INFO : optimized alpha [3.9300597, 0.2673792, 0.363505, 3.7168365, 3.9572504, 0.2946499, 0.4032809, 0.16954091, 4.4436994, 0.11378796, 0.2687835, 0.49217385, 0.14364743]
2020-03-14 21:28:15,333 : DEBUG : updating topics
2020-03-14 21:28:15,334 : INFO : merging changes from 500 documents into a model of 10000 documents
2020-03-14 21:28:15,337 : INFO : topic #9 (0.114): 0.207*"carne_asada" + 0.095*"pleasant" + 0.086*"carne" + 0.075*"asada" + 0.059*"platter" + 0.054*"environment" + 0.053*"carrot" + 0.050*"cover" + 0.049*"satisfied" + 0.049*"soggy"
2020-03-14 21:28:15,338 : INFO : topic #12 (0.144): 0.191*"staff_friendly" + 0.118*"hotel" + 0.110*"saturday_nig

2020-03-14 21:28:15,710 : INFO : topic #0 (3.939): 0.028*"restaurant" + 0.026*"which" + 0.022*"chicken" + 0.021*"ordered" + 0.019*"salad" + 0.018*"cheese" + 0.018*"sauce" + 0.017*"lunch" + 0.017*"delicious" + 0.015*"fresh"
2020-03-14 21:28:15,711 : INFO : topic #4 (3.974): 0.107*"place" + 0.081*"there" + 0.049*"really" + 0.027*"thing" + 0.026*"pretty" + 0.025*"little" + 0.025*"price" + 0.024*"other" + 0.024*"drink" + 0.023*"think"
2020-03-14 21:28:15,711 : INFO : topic #8 (4.475): 0.038*"about" + 0.037*"would" + 0.021*"after" + 0.019*"first" + 0.018*"could" + 0.017*"table" + 0.016*"friend" + 0.016*"because" + 0.015*"night" + 0.015*"again"
2020-03-14 21:28:15,712 : INFO : topic diff=0.010213, rho=0.091287
2020-03-14 21:28:15,715 : INFO : PROGRESS: pass 99, at document #8000/10000
2020-03-14 21:28:15,716 : DEBUG : performing inference on a chunk of 500 documents
2020-03-14 21:28:15,788 : DEBUG : 500/500 documents converged within 1000 iterations
2020-03-14 21:28:15,791 : INFO : optimized

2020-03-14 21:28:16,147 : DEBUG : updating topics
2020-03-14 21:28:16,149 : INFO : merging changes from 500 documents into a model of 10000 documents
2020-03-14 21:28:16,151 : INFO : topic #9 (0.114): 0.178*"carne_asada" + 0.100*"pleasant" + 0.075*"carne" + 0.068*"cover" + 0.065*"asada" + 0.060*"environment" + 0.059*"recent" + 0.058*"soggy" + 0.053*"relaxing" + 0.051*"satisfied"
2020-03-14 21:28:16,152 : INFO : topic #12 (0.145): 0.183*"staff_friendly" + 0.117*"hotel" + 0.114*"saturday_night" + 0.087*"downtown" + 0.068*"staff" + 0.058*"night" + 0.049*"stayed" + 0.048*"room" + 0.046*"friendly" + 0.045*"saturday"
2020-03-14 21:28:16,153 : INFO : topic #0 (3.948): 0.029*"restaurant" + 0.026*"which" + 0.023*"chicken" + 0.022*"ordered" + 0.019*"salad" + 0.018*"cheese" + 0.018*"sauce" + 0.017*"delicious" + 0.016*"lunch" + 0.016*"fresh"
2020-03-14 21:28:16,154 : INFO : topic #4 (3.960): 0.109*"place" + 0.081*"there" + 0.048*"really" + 0.026*"thing" + 0.026*"pretty" + 0.025*"price" + 0.025*"dr

Wall time: 3min 31s


## Technique 1 for Determining Optimal Number of Topics: Topic Coherence

Topic Coherence is a measure used to evaluate topic models. Each such generated topic consists of words, and the topic coherence is applied to the top N words from the topic. 

Topic Coherence measures score a single topic by **measuring the degree of semantic similarity between high scoring words in the topic**. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference. 

A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. An example of a coherent fact set is “the game is a team sport”, “the game is played with a ball”, “the game demands great physical efforts”

Topic Coherence is defined as the average of the pairwise word-similarity scores of the words in the topic.

A good model will generate coherent topics, i.e., topics with high topic coherence scores. Good topics are topics that can be described by a short label, therefore this is what the topic coherence measure should capture.


Below we display 
- the average topic coherence and
- print the topics in order of topic coherence


We use LdaModel's "top_topics" method to get the topics with the highest coherence score the coherence for each topic.

Note that we use the “Umass” topic coherence measure here (see gensim.models.ldamodel.LdaModel.top_topics()).

In [14]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2020-03-14 21:28:16,167 : DEBUG : Setting topics to those of the model: LdaModel(num_terms=1028, num_topics=13, decay=0.5, chunksize=500)
2020-03-14 21:28:16,189 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2020-03-14 21:28:16,207 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2020-03-14 21:28:16,226 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2020-03-14 21:28:16,243 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2020-03-14 21:28:16,261 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2020-03-14 21:28:16,279 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2020-03-14 21:28:16,296 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2020-03-14 21:28:16,314 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2020-03-14 21:28:16,332 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2020-03-14 21:28:16,349 : INFO : CorpusAccumulator accumulate

Average topic coherence: -4.8269.
[([(0.03832982, 'about'),
   (0.035716947, 'would'),
   (0.021039262, 'after'),
   (0.019128978, 'first'),
   (0.017565237, 'could'),
   (0.016728085, 'table'),
   (0.016140616, 'friend'),
   (0.015941052, 'because'),
   (0.015606949, 'night'),
   (0.015216036, 'staff'),
   (0.01426303, 'again'),
   (0.0139756985, 'experience'),
   (0.013949287, 'order'),
   (0.013943648, 'going'),
   (0.01365854, 'before'),
   (0.013226189, 'right'),
   (0.011998311, 'review'),
   (0.011465708, 'another'),
   (0.010933864, 'minute'),
   (0.010469571, 'while')],
  -1.743448821343539),
 ([(0.10855517, 'place'),
   (0.08102871, 'there'),
   (0.04848793, 'really'),
   (0.026217224, 'thing'),
   (0.025602343, 'pretty'),
   (0.024961986, 'price'),
   (0.024865229, 'drink'),
   (0.024734378, 'other'),
   (0.024679704, 'little'),
   (0.022512639, 'think'),
   (0.022222564, 'people'),
   (0.021660719, 'better'),
   (0.015269627, 'though'),
   (0.013275191, 'something'),
   (0.

## Technique 2 for Determining Optimal Number of Topics: Visualization

We use **pyLDAvis** to interpret the topics in a topic model that has been fit to a corpus of text data. 

It extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

In [15]:
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning) 

# pyLDAvis.gensim.prepare(model, corpus, dictionary)


## Interpretation of the Visualization 



- Left Panel: 
The labeld Intertopic Distance Map, circles represent different topics and the distance between them. Similar topics appear closer and the dissimilar topics farther. The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus. An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.



- Right Panel:
It includes the bar chart of the top 30 terms. When no topic is selected in the plot on the left, the bar chart shows the top-30 most "salient" terms in the corpus. A term's saliency is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics. Selecting each topic on the right, modifies the bar chart to show the "relevant" terms for the selected topic. 

Relevence is defined as in footer 2 and can be tuned by parameter $\lambda$.
- Smaller $\lambda$ gives higher weight to the term's distinctiveness.
- larger $\lambda$ corresponds to probablity of the term occurance per topics.

Therefore, to get a better sense of terms per topic we use $\lambda = 0$.

## Display the Top Words in the Topics

In [16]:
def get_lda_topics(model, num_topics, top_words):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = top_words);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [17]:
get_lda_topics(model, num_topics, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13
0,restaurant,recommend,sandwich,great,place,customer_service,store,mexican,about,carne_asada,sushi,money,staff_friendly
1,which,highly_recommend,burger,their,there,customer,selection,taco,would,pleasant,bartender,watch,hotel
2,chicken,friday_night,fry,service,really,reasonably_priced,item,chip,after,carne,roll,market,saturday_night
3,ordered,highly,breakfast,always,thing,priced,helpful,salsa,first,cover,crowd,event,downtown
4,salad,friday,sweet_potato,pizza,pretty,employee,dining_experience,burrito,could,asada,beer,mashed_potato,staff
5,cheese,margarita,potato,friendly,price,across_street,italian,chip_salsa,table,environment,spring,movie,night
6,sauce,class,bacon,location,drink,across,waffle,bean,friend,recent,sport,birthday,stayed
7,delicious,course,reasonable,time,other,company,bought,tortilla,because,soggy,cocktail,bathroom,room
8,lunch,seafood,french,every,little,phone,section,airport,night,relaxing,drinking,spend,friendly
9,fresh,suggest,price_reasonable,favorite,think,closed,professional,enchilada,staff,satisfied,resort,buck,saturday


## Generate Labels for the Topics

We can manually generate human-interpretable labels for each topic by looking at the terms that appear more in each topic.


We use LdaModel's "show_topic" method that returns **Word-probability pairs** for the most relevant words generated by the topic.

In [18]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, a topic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:30} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [19]:
topic_summaries = []

print(u'{:25} {}'.format(u'term', u'frequency') + u'\n')

for i in range(num_topics):
    print('\nTopic '+str(i)+' |---------------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                      frequency


Topic 0 |---------------------------

restaurant                     0.029
which                          0.026
chicken                        0.023
ordered                        0.022
salad                          0.019
cheese                         0.018
sauce                          0.018
delicious                      0.017
lunch                          0.016
fresh                          0.016

Topic 1 |---------------------------

recommend                      0.153
highly_recommend               0.136
friday_night                   0.079
highly                         0.074
friday                         0.072
margarita                      0.055
class                          0.051
course                         0.032
seafood                        0.032
suggest                        0.029

Topic 2 |---------------------------

sandwich                       0.152
burger                         0.147
fry                            0

## Manually Generate Topic Labels

Based on the most probable words generated by each topic, we assign human-interpretable labels for the topics.

In [20]:
top_labels = {0: 'Statistics', 1:'Numerical Analysis', 2:'Online Learning', 3:'Deep Learning'}